In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [15]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize

import numpy as np
from utils import *
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
def predict(text, model, batch_size=5):
    model.eval()
    sentences = sent_tokenize(text)
    label_list = ['positive', 'negative', 'neutral']
    label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}
    device = "cpu"
    result = pd.DataFrame(columns=['sentence', 'logit', 'prediction', 'sentiment_score'])
    for batch in chunks(sentences, batch_size):
        examples = [InputExample(str(i), sentence) for i, sentence in enumerate(batch)]
        features = convert_examples_to_features(examples, label_list, 64, tokenizer)

        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long).to(device)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long).to(device)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long).to(device)

        with torch.no_grad():
            model     = model.to(device)

            logits = model(all_input_ids, all_attention_mask, all_token_type_ids)[0]
            logging.info(logits)
            logits = softmax(np.array(logits.cpu()))
            sentiment_score = pd.Series(logits[:, 0] - logits[:, 1])
            predictions = np.squeeze(np.argmax(logits, axis=1))

            batch_result = {'sentence': batch,
                            'logit': list(logits),
                            'prediction': predictions,
                            'sentiment_score': sentiment_score}

            batch_result = pd.DataFrame(batch_result)
            result = pd.concat([result, batch_result], ignore_index=True)

    result['prediction'] = result.prediction.apply(lambda x: label_dict[x])
    return result

In [4]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [18]:
result = predict(text,model)  
print(result)
print(np.mean(result.sentiment_score))

06/05/2022 00:21:03 - INFO - utils -   *** Example ***
06/05/2022 00:21:03 - INFO - utils -   guid: 0
06/05/2022 00:21:03 - INFO - utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
06/05/2022 00:21:03 - INFO - utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
06/05/2022 00:21:03 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
06/05/2022 00:21:03 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
06/05/20

                                            sentence  \
0  Later that day Apple said it was revising down...   
1       The news rapidly infected financial markets.   
2  Apple’s share price fell by around 7% in after...   
3  The dollar fell by 3.7% against the yen in a m...   
4  Asian stockmarkets closed down on January 3rd ...   
5  Yields on government bonds fell as investors f...   

                                     logit prediction sentiment_score  
0  [0.0074973884, 0.96461785, 0.027884703]   negative        -0.95712  
1    [0.017176522, 0.87915826, 0.10366525]   negative       -0.861982  
2  [0.0066501475, 0.97392964, 0.019420119]   negative       -0.967279  
3     [0.050482538, 0.9313251, 0.01819236]   negative       -0.880843  
4    [0.0068846154, 0.972121, 0.020994334]   negative       -0.965236  
5   [0.009239607, 0.96016717, 0.030593174]   negative       -0.950928  
-0.9305647015571594


In [1]:
from keybert_api import KeyBERTWrapper

In [3]:
keybert_model = KeyBERTWrapper()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

NameError: name 'text' is not defined

In [6]:
keybert_model.predict(text)[0]

('markets apple', 0.5656)

In [7]:
import plotly.express as px
import numpy as np
from collections import deque
import pandas as pd
sentiment_q = deque()
keyword_q = deque()

keyword_q.append("tesla")
keyword_q.append('musk')
sentiment_q.append(1)
sentiment_q.append(-1)
keyword_q.append("cat")
sentiment_q.append(0.2)

df = pd.DataFrame(dict(sentiment=sentiment_q,keyword=keyword_q, index=[i for i in range(len(sentiment_q))]))
print(df)
fig = px.treemap(df, path=["keyword"], values='sentiment',
                  color='sentiment',
                  color_continuous_scale='RdYlGn',
                  color_continuous_midpoint=0)
                  
fig.show()

   sentiment keyword  index
0        1.0   tesla      0
1       -1.0    musk      1
2        0.2     cat      2


In [ ]:
import requests 
text = 'hellow im happy to chat'
r_sentiment = requests.get(url = "/".join([url, "sentiment"]), data = text.encode('utf-8'))